In [13]:
#If the code is running on Google Colab, I need to mount the GDrive into the workspace to import my homemade dependencies
google_colab = True
if google_colab:
    #Mount google drive into google colab working space
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    #Copy my functions into Colab workspace
    !cp "/content/drive/MyDrive/Colab Notebooks/Federated Learning/models.py" .

Mounted at /content/drive


In [14]:
#Imports
import models
import numpy as np
import time
import tensorflow as tf
from matplotlib import pyplot as plt

In [15]:
#Simulation Variables
K = 10
local_epochs = 1
steps_per_epoch = None #None = iterate until entire dataset is covered once
N = 10 #Number of communication rounds

In [16]:
#Download global dataset and split it into K local datasets
local_datasets, global_dataset = models.load_dataset(K)
#CIFAR10 labels
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
#Create K local models and one global model with randomly initialized weights
model_list, global_model = models.setup_models(K, len(labels))

In [17]:
#Federated learning loop
start = time.time()
for i in range(N):
    print("Communication round " + str(i+1) + ":")
    #Broadcast to devices
    model_list, global_model = models.model_broadcast(model_list, global_model)

    #Local training
    model_list = models.local_training(model_list, local_datasets, local_epochs, steps_per_epoch)

    #Global Aggregation
    model_list, global_model = models.global_update(model_list, global_model)
end = time.time()
print("Time spent on Federated Learning =", end-start)

Communication round 0:
79/79 [==============================] - 1s 8ms/step - loss: 2.1739 - accuracy: 0.1858
Communication round 1:
79/79 [==============================] - 1s 8ms/step - loss: 1.9657 - accuracy: 0.2798
Communication round 2:
79/79 [==============================] - 1s 8ms/step - loss: 1.7345 - accuracy: 0.3604
Communication round 3:
79/79 [==============================] - 1s 8ms/step - loss: 1.5774 - accuracy: 0.4162
Communication round 4:
79/79 [==============================] - 1s 8ms/step - loss: 1.5151 - accuracy: 0.4364
Communication round 5:
79/79 [==============================] - 1s 9ms/step - loss: 1.3869 - accuracy: 0.4938
Communication round 6:
79/79 [==============================] - 1s 8ms/step - loss: 1.3028 - accuracy: 0.5258
Communication round 7:
79/79 [==============================] - 1s 8ms/step - loss: 1.2241 - accuracy: 0.5572
Communication round 8:
79/79 [==============================] - 1s 8ms/step - loss: 1.1429 - accuracy: 0.6010
Communicat

In [19]:
#Accuracy evaluation 
results = global_model.evaluate(global_dataset[0], global_dataset[1], batch_size=128, verbose=0) 
print("train loss, train acc of global model:", results) 
results = global_model.evaluate(global_dataset[2], global_dataset[3], batch_size=128, verbose=0) 
print("test loss, test acc of global model:", results) 

train loss, train acc of global model: [0.984465479850769, 0.6495800018310547]
test loss, test acc of global model: [1.022385835647583, 0.6338000297546387]
train loss, train acc of local model 0: [1.1613014936447144, 0.578719973564148]
test loss, test acc of local model 0: [1.1905784606933594, 0.5712000131607056]
